# Comparison with article

In [1]:
from coupled_cluster.schemes.ccd_sparse import CoupledClusterDoublesSparse
from coupled_cluster.hartree_fock.scf_rhf import scf_rhf
from coupled_cluster.matrix_elements.generate_matrices import (
    get_one_body_elements, get_coulomb_elements,
    get_antisymmetrized_elements, add_spin_to_one_body_elements,
    get_one_body_elements_spin
)
from coupled_cluster.hartree_fock.basis_transformation import (
    transform_one_body_elements, transform_two_body_elements
)

import numpy as np
import time

In [2]:
omega = 0.01
l = 30
n = 2

print ("""
w = {0},
l = {1},
n = {2}
""".format(omega, l, n))


w = 0.01,
l = 30,
n = 2



In [3]:
h = omega * get_one_body_elements(l)
t0 = time.time()
u = np.sqrt(omega) * get_coulomb_elements(l)
t1 = time.time()
print ("Time spent creating Coulomb elements: {0} sec".format(t1 - t0))

t0 = time.time()
c, energy = scf_rhf(h.todense(), u, np.eye(l//2), n//2)
t1 = time.time()
print ("Time spent in SCF RHF: {0} sec".format(t1 - t0))

print ("\tRHF Energy: {0}".format(energy))

hi = transform_one_body_elements(h, c)
t0 = time.time()
oi = transform_two_body_elements(u, c)
t1 = time.time()
print ("Time spent transforming two body elements: {0} sec".format(t1 - t0))

_h = add_spin_to_one_body_elements(hi, l)
t0 = time.time()
_u = get_antisymmetrized_elements(l, oi=oi)
t1 = time.time()
print ("Time spent antisymmetrizing two body elements: {0} sec".format(t1 - t0))

ccd_hf = CoupledClusterDoublesSparse(_h, _u, n)
t0 = time.time()
print ("\tCCD (HF) Energy: {0}".format(ccd_hf.compute_energy()))
t1 = time.time()
print ("Time spent computing CCD energy with HF basis: {0} sec".format(t1 - t0))

__h = omega * get_one_body_elements_spin(l)
t0 = time.time()
__u = np.sqrt(omega) * get_antisymmetrized_elements(l)
t1 = time.time()
print ("Time spent getting antisymmetric two body elements: {0} sec".format(t1 - t0))

ccd = CoupledClusterDoublesSparse(__h, __u, n)
t0 = time.time()
print ("\tCCD Energy: {0}".format(ccd.compute_energy()))
t1 = time.time()
print ("Time spent computing CCD energy with HO basis: {0} sec".format(t1 - t0))

Time spent creating Coulomb elements: 0.20420026779174805 sec
Time spent in SCF RHF: 0.018474340438842773 sec
	RHF Energy: 0.10832124612503626
Time spent transforming two body elements: 0.004179477691650391 sec
Time spent antisymmetrizing two body elements: 0.03146052360534668 sec
	CCD (HF) Energy: 0.07713741507385612
Time spent computing CCD energy with HF basis: 0.48984289169311523 sec
Time spent getting antisymmetric two body elements: 0.20566034317016602 sec
	CCD Energy: 0.09516040906297069
Time spent computing CCD energy with HO basis: 1.3820774555206299 sec


In [4]:
print (h.density)
print (u.density)
print (_u.density)
print (_h.density)
print (__u.density)
print (__h.density)

0.06666666666666667
0.08452345679012346
0.12331358024691358
0.07777777777777778
0.030503703703703703
0.03333333333333333
